In [1]:
import requests, operator, pandas, glob2
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [2]:
# 크롤링 함수(날짜와 페이지수를 입력받아 그날짜의 그 페이지수만큼 크롤링 해옴)
def crawlingData(date, pageCount):
    # 현재 시각을 now라는 변수에 저장
    now = datetime.now()
    l = [] # 리스트 l

    # pagecount는 1페이지부터 사용자가 입력한 페이지 수까지 됨
    for pagecount in range(1, int(pageCount)+1):

        # 동적으로, 사용자가 입력한 날짜와 뉴스 페이지에 접속
        r = requests.get("http://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100&date=" + str(date) + "&page=" + str(pagecount))
        c = r.content
        soup = BeautifulSoup(c, "html.parser")
        all = soup.find_all("li")

        for item in all:
            for item2 in item.find_all("dl"):
                d = {} # 사전 d
                try:
                    linkTag = item2.find("dt", {"class": ""}).find("a")
                    d["LinkSrc"] = linkTag['href'] # 사전 d의 LinkSrc라는 키에 href 내용을 가져와 저장
                    title = linkTag.text.replace("\t", "").replace("\n", "").replace(",", "").replace('"',"").replace("\r", "")
                    title = title [1:len(title.text) + 1]
                    d["Title"] = title
                except:
                    d["LinkSrc"] = "None"
                    d["Title"] = "None"

                try:
                    contentTag = item2.find("dd")
                    d["Content"] = \
                    contentTag.text.replace("\t", "").replace("\n", "").replace("\r", "").replace(",", "").replace('"',"").split("…")[0]
                    d["Company"] = contentTag.find("span", {"class": "writing"}).text
                    d["Date"] = contentTag.find("span", {"class": "date"}).text
                    #print(d["Content"])
                except:
                    d["Content"] = "None"
                    d["Company"] = "None"
                    d["Date"] = "None"

                try:
                    imgTag = item2.find("dt", {"class": "photo"}).find("img")
                    d["imgSrc"] = imgTag["src"]
                except:
                    d["imgSrc"] = "No image"

                l.append(d) # 리스트에 사전 추가 / 한 행마다 사전에 추가

    df = pandas.DataFrame(l) # pandas 사용 l의 데이터프레임화

    # now(현재시각)을 이용해 csv 파일로 저장
    df.to_csv('%s-%s-%s-%s-%s-%s.csv' % (now.year, now.month, now.day, now.hour, now.minute, now.second),
              encoding='utf-8-sig', index=False)
    print("Success Get DataFIle and Save Data")


In [26]:
crawlingData(20190701, 10)

Success Get DataFIle and Save Data


In [3]:
# ppt. 523 loadFile 함수

def loadFile(fileName):
    # checkFileName 함수를 호출, 파일이 존재하는지 확인
    outputFileName = checkFileName(fileName)
    
    if outputFileName is not -1:
        df = pd.read_csv(outputFileName)
        content = df["Content"]
        title = df["Title"]
        company = df["Company"]
        print(company)
        print("csv File Load Success")
    else:
        print("Error csv File")

In [4]:
# ppt. 524 checkFileName

def checkFileName(fileName):
    now = datetime.now()
    
    # 같은 경로에 csv 파일 하나도 없으면 -1 리턴
    if len(glob2.glob("*.csv")) == 0:
        print("No file found in this directory")
        return -1
    else:
        if fileName == "all":   # 사용자가 입력한 값이 all 이라면
            result = []
            # for문 돌며 모든 csv 파일 가져와 읽은 후 result 리스트에 저장
            for i in glob2.glob("*.csv"):
                result.append(pd.read_csv(i))
            
            # 새로 만들 파일 이름 지정
            outputFileName = '%s-%s-%s-%s-%s-%s merging.csv' %(
                now.year, now.month, now.day, now.hour, now.minute, now.second)
            
            # 리스트에 저장한 csv 파일들을 resultDf 변수에 저장
            resultDf = pd.concat(result, ignore_index = True)
            resultDf.to_csv(outputFileName, encoding='utf-8-sig') # csv 파일 생성
            return outputFileName
        else:                   # 사용자가 입력한 값이 all이 아니라면
            return fileName

In [5]:
# 메인 세팅 함수, 사용자로부터 값을 입력받아 함수를 호출
def mainSetting():
    while(1):
        kb = input("$ ")
        if kb == "exit":
            break
        elif kb == "crawling":
            date = input("Enter news date : ")
            page = input("Enter your pageCount : ")
            crawlingData(date, page)
        elif kb == "loadAll":
            loadFile("all")
        elif kb == "load":
            fileName = input("Enter your csv file name : ")
            loadFile(fileName)
        else:
            print("command error")


mainSetting()

$ crawling
Enter news date : 20000101
Enter your pageCount : 30
Success Get DataFIle and Save Data
$ 20010101
command error
$ crawling
Enter news date : 20010101
Enter your pageCount : 30
Success Get DataFIle and Save Data
$ crawling
Enter news date : 20100101
Enter your pageCount : 30
Success Get DataFIle and Save Data
$ 20110101
command error
$ crawling
Enter news date : 20110101
Enter your pageCount : 30
Success Get DataFIle and Save Data
$ exit


In [11]:
# while(True)는 반복을 위해 있기 때문에, while과 break를 지워도 (한 번만) 실행된다.
def mainSetting():
    kb = input("$ ")
    if kb == "exit":
        pass
    elif kb == "crawling":
        date = input("Enter news date : ")
        page = input("Enter your pageCount : ")
        crawlingData(date, page)
    elif kb == "loadAll":
        loadFile("all")
    elif kb == "load":
        fileName = input("Enter your csv file name : ")
        loadFile(fileName)
    else:
        print("command error")


mainSetting()

$ crawling
Enter news date : 20190101
Enter your pageCount : 1
Success Get DataFIle and Save Data


In [ ]:
checkFileName(20190101)

In [7]:
checkFileName("all")

C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



'2019-7-12-16-23-36 merging.csv'

In [25]:
loadFile("2019-7-12-14-9-13.csv")

0      아이뉴스24
1      한국경제TV
2        노컷뉴스
3        서울경제
4         YTN
5         KBS
6      아이뉴스24
7         뉴시스
8        이데일리
9         KBS
10        뉴시스
11        뉴시스
12        뉴시스
13        SBS
14        YTN
15        YTN
16        YTN
17        YTN
18        YTN
19        YTN
20        YTN
21       세계일보
22       JTBC
23       매일경제
24     파이낸셜뉴스
25       JTBC
26       경향신문
27       경향신문
28        뉴시스
29        KBS
        ...  
90        KBS
91     파이낸셜뉴스
92       데일리안
93       TV조선
94       TV조선
95      오마이뉴스
96       TV조선
97       세계일보
98       데일리안
99        뉴스1
100       뉴스1
101       뉴스1
102       뉴스1
103       뉴스1
104       뉴스1
105       뉴스1
106       뉴스1
107       뉴스1
108       뉴스1
109       뉴스1
110       뉴스1
111       뉴스1
112       뉴스1
113       뉴스1
114       뉴스1
115      데일리안
116       뉴시스
117      데일리안
118      데일리안
119       뉴스1
Name: Company, Length: 120, dtype: object
csv File Load Success


20190101

In [6]:
now = datetime.now()

In [7]:
now

datetime.datetime(2019, 7, 12, 13, 28, 51, 875141)

In [23]:
age = input()
age + 1

20


TypeError: Can't convert 'int' object to str implicitly

In [24]:
int(age)+1

21

In [ ]:
https://news.naver.com/main/list.nhn?%20mode=LSD&mid=sec&sid1=100
https://news.naver.com/main/list.nhn?%20mode=LSD&mid=sec&sid1=100&date=20190701&page=2